In [1]:
'''
This script is used to decode the generated latent embedding into the gene expression matrix and evaluate the realistic of generated cells
Be careful to change the paths that appear in the file to your local path
'''

'\nThis script is used to decode the generated latent embedding into the gene expression matrix and evaluate the realistic of generated cells\nBe careful to change the paths that appear in the file to your local path\n'

In [2]:
import anndata as ad
import scanpy as sc
import matplotlib.pyplot as plt
import numpy as np
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import pandas as pd
from scipy import stats
import torch
import sys
sys.path.append('..')
from VAE.VAE_model import VAE
from torch.autograd import Variable
import celltypist
from tqdm import tqdm

/home/yjh/anaconda3/envs/scld/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/yjh/anaconda3/envs/scld/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/home/yjh/anaconda3/envs/scld/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/home/yjh/anaconda3/envs/scld/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/home/yjh/anaconda3/envs/scld/lib/python3.10/site-packages/anndata/utils.py:429: 

load AE. Change the path of autoencoder to your local path

In [3]:
def load_VAE():
    autoencoder = VAE(
        num_genes=18996,
        device='cuda',
        seed=0,
        loss_ae='mse',
        hidden_dim=128,
        decoder_activation='ReLU',
    )
    autoencoder.load_state_dict(torch.load('../data/muris_AE/model_seed=0_step=199999.pt'))
    return autoencoder

load real data for comparision

In [4]:
adata = sc.read_h5ad('../data/muris/data/tabula_muris/all.h5ad')
adata.var_names_make_unique()
sc.pp.filter_cells(adata, min_genes=10)
sc.pp.filter_genes(adata, min_cells=3)
gene_names = adata.var_names

sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
cell_data = adata.X.toarray()[::5]

cell_data.shape

(11401, 18996)

load and decode unconditional generated data

In [5]:
# the generated data path

'''npzfile=np.load('../samples/补充实验/LISI/scDiffusion/muris0.npz',allow_pickle=True)
cell_gen_all = npzfile['cell_gen'][:10000]'''
npzfile=np.load('../samples/补充实验/LISI/ESCFD/muris0.npz',allow_pickle=True)
cell_gen_all = npzfile['samples'][:10000]

autoencoder = load_VAE()
cell_gen_all = autoencoder(torch.tensor(cell_gen_all).cuda(),return_decoded=True).detach().cpu().numpy()
ori = ad.AnnData(cell_gen_all, dtype=np.float32)
cell_gen = ori.X
cell_gen.shape
import scib
adata_scib = np.concatenate((cell_data, cell_gen),axis=0)
adata_scib = ad.AnnData(adata_scib, dtype=np.float32)
adata_scib.obs['batch'] = pd.Categorical([f"true_Cell" for i in range(cell_data.shape[0])]+[f"gen_Cell" for i in range(cell_gen.shape[0])])
sc.pp.neighbors(adata_scib, n_neighbors=10, n_pcs=20)
scib.me.ilisi_graph(adata_scib, batch_key="batch", type_="knn")


/home/yjh/anaconda3/envs/scld/lib/python3.10/site-packages/anndata/_core/anndata.py:401: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/yjh/anaconda3/envs/scld/lib/python3.10/site-packages/scanpy/tools/_utils.py:41: UserWarning: You’re trying to run this on 18996 dimensions of `.X`, if you really want this, set `use_rep='X'`.
         Falling back to preprocessing with `sc.pp.pca` and default params.
  warnings.warn(


0.8843402406110863

calculate correlation

In [6]:
print('spearman=',stats.spearmanr(cell_data.mean(axis=0), cell_gen.mean(axis=0)).correlation)
print('pearson=',np.corrcoef(cell_data.mean(axis=0), cell_gen.mean(axis=0))[0][1])

spearman= 0.9938733255829303
pearson= 0.9994150878414704
